In [3]:
import os
import requests
from pandas import DataFrame, option_context
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('EXCHANGE_RATE_API_KEY')

In [4]:
base_currency = "USD"
url_currency_rates = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}"
req_currency = requests.get(url_currency_rates)

data = req_currency.json()

# Extract conversion rates and create DataFrame
conversion_rates = DataFrame(data['conversion_rates'].items(), columns=['Currency_Code', 'Rate'])

# Sort DataFrame by Rate in descending order
highest_rates = conversion_rates.sort_values(by="Rate", ascending=False).reset_index(drop=True)
highest_rates["date"] = data["time_last_update_utc"].split(" 00:")[0]
highest_rates.head()

,Currency_Code,Rate,date
0,LBP,89500.0000,"Sun, 02 Jun 2024"
1,IRR,42063.1818,"Sun, 02 Jun 2024"
2,VND,25469.2457,"Sun, 02 Jun 2024"
3,SLL,22504.0936,"Sun, 02 Jun 2024"
4,LAK,21655.8461,"Sun, 02 Jun 2024"


- Add country names from this site (quick scraping) https://www.foreignexchangelive.com/currency-codes-symbols/

In [5]:
url_currency_codes = f'https://v6.exchangerate-api.com/v6/{api_key}/codes'
req_codes = requests.get(url_currency_codes)
data_code = req_codes.json()
codes = DataFrame(data_code["supported_codes"], columns=["Currency_Code", "Currency_Name"])

In [6]:
import pandasql as ps
query = """
SELECT Currency_Code, Rate, Currency_Name, date
FROM highest_rates NATURAL JOIN codes
"""
result = ps.sqldf(query, locals())

In [7]:
with option_context('display.max_rows', None, 'display.max_columns', None):
    display(result)

,Currency_Code,Rate,Currency_Name,date
0,LBP,89500.0000,Lebanese Pound,"Sun, 02 Jun 2024"
1,IRR,42063.1818,Iranian Rial,"Sun, 02 Jun 2024"
2,VND,25469.2457,Vietnamese Đồng,"Sun, 02 Jun 2024"
3,SLL,22504.0936,Sierra Leonean Leone,"Sun, 02 Jun 2024"
4,LAK,21655.8461,Lao Kip,"Sun, 02 Jun 2024"
5,IDR,16248.8365,Indonesian Rupiah,"Sun, 02 Jun 2024"
6,SYP,12903.8830,Syrian Pound,"Sun, 02 Jun 2024"
7,UZS,12668.2234,Uzbekistani So'm,"Sun, 02 Jun 2024"
8,GNF,8584.4392,Guinean Franc,"Sun, 02 Jun 2024"
9,PYG,7511.8283,Paraguayan Guaraní,"Sun, 02 Jun 2024"


In [8]:
import sqlite3

In [13]:
import sqlite3
# Create a sqlite3 database
conn = sqlite3.connect('currency_rates.db')
c = conn.cursor()

# Push the data to the database
result.to_sql('currency', conn, if_exists='replace', index=False)

query = """
SELECT *
FROM currency
"""
import pandas as pd
old_data = pd.read_sql_query(query, conn)
new_data = pd.concat([old_data, result], ignore_index=True)

In [15]:
query = """
SELECT *
FROM currency
"""
import pandas as pd
old_data = pd.read_sql_query(query, conn)

In [16]:
old_data

,Currency_Code,Rate,Currency_Name,date
0,LBP,89500.0000,Lebanese Pound,"Sun, 02 Jun 2024"
1,IRR,42063.1818,Iranian Rial,"Sun, 02 Jun 2024"
2,VND,25469.2457,Vietnamese Đồng,"Sun, 02 Jun 2024"
3,SLL,22504.0936,Sierra Leonean Leone,"Sun, 02 Jun 2024"
4,LAK,21655.8461,Lao Kip,"Sun, 02 Jun 2024"
...,...,...,...,...
157,XDR,0.7552,Special Drawing Rights,"Sun, 02 Jun 2024"
158,JOD,0.7090,Jordanian Dinar,"Sun, 02 Jun 2024"
159,OMR,0.3845,Omani Rial,"Sun, 02 Jun 2024"
160,BHD,0.3760,Bahraini Dinar,"Sun, 02 Jun 2024"
